In [42]:
from math import sqrt
from scipy import stats

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pydataset import data
import statistics
import acquire
import prepare

- id_vars = columns you want to keep (not melt)
- var_name = name of new column you created by melting columns
- value_name = column name for values

# 1.Attendance Data

### Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

### You should end up with something like this:


name
Billy    0.5250

Jane     0.6875

John     0.9125

Sally    0.7625

Name: grade, dtype: float64

In [2]:
attendance = pd.read_csv('untidy-data/attendance.csv')
attendance

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attendance = attendance.rename(columns = {'Unnamed: 0' : 'student_name'})
attendance.head(5)

,student_name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [4]:
attendance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   student_name  4 non-null      object
 1   2018-01-01    4 non-null      object
 2   2018-01-02    4 non-null      object
 3   2018-01-03    4 non-null      object
 4   2018-01-04    4 non-null      object
 5   2018-01-05    4 non-null      object
 6   2018-01-06    4 non-null      object
 7   2018-01-07    4 non-null      object
 8   2018-01-08    4 non-null      object
dtypes: object(9)
memory usage: 416.0+ bytes


In [5]:
attendance.shape

(4, 9)

In [7]:
attendance_melt = pd.melt(attendance, id_vars=['student_name'], var_name ='date', value_name = 'value')
attendance_melt.head()


,student_name,date,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [10]:
attendance_melt.value.unique()

array(['P', 'A', 'T', 'H'], dtype=object)

P = 'present' = 100%
A = 'absent' = 0%
(10 tardies is equal to one absence)
T = 'tardy' = 90%
H = 'half day' = 50%

In [121]:
def grade(value):
    if value == 'P':
        return 1
    elif value == 'A':
        return 0
    elif value == 'T':
        return .9
    elif value == 'H':
        return .5



In [ ]:
# can do a replace function too
attendance_clean = attendance_melt.replace({'P':1, 'T': .})


In [120]:
attendance_melt['grade'] = attendance_melt.value.apply(grade)

In [24]:
attendance_melt.head()

,student_name,date,value,grade
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9


In [33]:
attendance_melt['student_name'].value_counts()

Sally    8
Jane     8
Billy    8
John     8
Name: student_name, dtype: int64

In [20]:
attendance_melt.groupby('student_name').grade.mean()

student_name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64

In [ ]:
#convert alphabets to numbers
#write a function takes the alphabets to a number or can do .replace
#calculate the mean grade for each student - think groupby

# 2. Coffee Levels

### - Read the coffee_levels.csv file.
### - Transform the data so that each carafe is in it's own column.
### - Is this the best shape for the data?

In [38]:
coffee = pd.read_csv('untidy-data/coffee_levels.csv')
coffee.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [39]:
coffee.shape

(30, 3)

In [41]:
coffee.coffee_carafe.value_counts()

x    10
y    10
z    10
Name: coffee_carafe, dtype: int64

In [45]:
coffee_melt = pd.melt (coffee, id_vars = 'coffee_carafe')#, var_name = 'amount', value_name = 'coffe_amount')
coffee_melt.head()

,coffee_carafe,variable,value
0,x,hour,8.0
1,x,hour,9.0
2,x,hour,10.0
3,x,hour,11.0
4,x,hour,12.0


In [50]:
coffee_tidy = coffee.pivot_table(index = ['hour'], columns = 'coffee_carafe',
                                values = 'coffee_amount').reset_index()
coffee_tidy.head()

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947


## 3. Cake Recipes

### -Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [90]:
cake = pd.read_csv('untidy-data/cake_recipes.csv')
cake.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


In [91]:
cake.shape

(8, 5)

### -Tidy the data as necessary.

In [92]:
cake['recipe:position'].value_counts()
#lets split recipe from position

a:bottom    1
a:top       1
b:bottom    1
b:top       1
c:bottom    1
c:top       1
d:bottom    1
d:top       1
Name: recipe:position, dtype: int64

In [93]:
cake['recipe:position'].str.split(':', expand = True)


,0,1
0,a,bottom
1,a,top
2,b,bottom
3,b,top
4,c,bottom
5,c,top
6,d,bottom
7,d,top


In [94]:
cake[['recipe', 'position']] =cake['recipe:position'].str.split(':', expand = True)
cake

,recipe:position,225,250,275,300,recipe,position
0,a:bottom,61.738655,53.912627,74.414730,98.786784,a,bottom
1,a:top,51.709751,52.009735,68.576858,50.228470,a,top
2,b:bottom,57.095320,61.904369,61.196980,99.248541,b,bottom
3,b:top,82.455004,95.224151,98.594881,58.169349,b,top
4,c:bottom,96.470207,52.001358,92.893227,65.473084,c,bottom
5,c:top,71.306308,82.795477,92.098049,53.960273,c,top
6,d:bottom,52.799753,58.670419,51.747686,56.183110,d,bottom
7,d:top,96.873178,76.101363,59.571620,50.971626,d,top


In [95]:
cake = cake.drop(columns = 'recipe:position')
cake

,225,250,275,300,recipe,position
0,61.738655,53.912627,74.414730,98.786784,a,bottom
1,51.709751,52.009735,68.576858,50.228470,a,top
2,57.095320,61.904369,61.196980,99.248541,b,bottom
3,82.455004,95.224151,98.594881,58.169349,b,top
4,96.470207,52.001358,92.893227,65.473084,c,bottom
5,71.306308,82.795477,92.098049,53.960273,c,top
6,52.799753,58.670419,51.747686,56.183110,d,bottom
7,96.873178,76.101363,59.571620,50.971626,d,top


In [ ]:
#now let's split

In [108]:
cake_melt = cake.melt(id_vars = ['recipe', 'position'], var_name = 'temp', value_name = 'value')
cake_melt

,recipe,position,temp,value
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207
5,c,top,225,71.306308
6,d,bottom,225,52.799753
7,d,top,225,96.873178
8,a,bottom,250,53.912627
9,a,top,250,52.009735


### -Which recipe, on average, is the best? recipe b

In [109]:
cake_melt['recipe'].value_counts()

a    8
b    8
c    8
d    8
Name: recipe, dtype: int64

In [124]:
cake_melt.groupby('recipe').value.mean().sort_values(ascending = False).head(1)

recipe
b    76.736074
Name: value, dtype: float64

In [123]:
cake_melt.groupby('recipe').value.mean().idxmax()

'b'

### -Which oven temperature, on average, produces the best results? 275


In [125]:
cake_melt.groupby('temp').value.mean().sort_values(ascending = False).head(1)

temp
275    74.886754
Name: value, dtype: float64

In [ ]:
#pivot table - index = recipe , columns = temp

### -Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees


In [131]:
cake_melt.groupby(['temp', 'recipe', 'position']).value.mean().sort_values(ascending = False).head(1)

temp  recipe  position
300   b       bottom      99.248541
Name: value, dtype: float64